<a href="https://colab.research.google.com/github/cxbxmxcx/EatNoEat/blob/master/Chapter_9_Data_Prep.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Imports


In [0]:
import tensorflow as tf
import matplotlib.pyplot as plt

# Scikit-learn includes many helpful utilities
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

import re
import numpy as np
import os
import time
import json
from glob import glob
from PIL import Image
import pickle

Download Recipe Data

In [10]:
data_folder = 'data'
recipes_zip = tf.keras.utils.get_file('recipes.zip',                                     
                                          origin = 'https://www.dropbox.com/s/i1hvs96mnahozq0/Recipes5k.zip?dl=1',
                                          extract = True)
print(recipes_zip)
data_folder = os.path.dirname(recipes_zip)
os.remove(recipes_zip)
print(data_folder)

164929536/164925728 [==============================] - 5s 0us/step
/root/.keras/datasets/recipes.zip
/root/.keras/datasets


Setup Folder Paths

In [11]:
!dir /root/.keras/datasets
data_folder = data_folder + '/Recipes5k/'
annotations_folder = data_folder + 'annotations/'
images_folder = data_folder + 'images/'
print(annotations_folder)
print(images_folder)

Recipes5k
/root/.keras/datasets/Recipes5k/annotations/
/root/.keras/datasets/Recipes5k/images/


In [12]:
%ls /root/.keras/datasets/Recipes5k/images/

apple_pie/            dumplings/                omelette/
baby_back_ribs/       edamame/                  onion_rings/
baklava/              eggs_benedict/            oysters/
beef_carpaccio/       escargots/                pad_thai/
beef_tacos/           falafel/                  paella/
beef_tartare/         filet_mignon/             pancakes/
beet_salad/           fish_and_chips/           panna_cotta/
beignets/             foie_gras/                peking_duck/
bibimbap/             french_fries/             pho/
bread_pudding/        french_onion_soup/        pizza/
breakfast_burrito/    french_toast/             pork_chop/
bruschetta/           fried_calamari/           poutine/
caesar_salad/         fried_rice/               prime_rib/
cannoli/              frozen_yogurt/            pulled_pork_sandwich/
caprese_salad/        garlic_bread/             ramen/
carrot_cake/          gnocchi/                  ravioli/
ceviche/              greek_salad/              red_velvet_cake/


Extra Imports

In [0]:
from fastprogress.fastprogress import master_bar, progress_bar
from IPython.display import Image
from os import listdir
from pickle import dump

In [0]:
def load_image(image_path):
  img = tf.io.read_file(image_path)
  img = tf.image.decode_jpeg(img, channels=3)
  img = tf.image.resize(img, (299, 299))
  img = tf.keras.applications.inception_v3.preprocess_input(img)
  return img, image_path

In [0]:
os.remove(foods_txt)

In [74]:
foods_txt = tf.keras.utils.get_file('foods.txt',
                        origin = 'https://www.dropbox.com/s/6jjje7mhqo9ycyi/foods.txt?dl=1')

print(foods_txt)

8192/2298 [==========================================================================================================] - 0s 0us/step
/root/.keras/datasets/foods.txt


In [0]:
def get_nutrient_array(fat, protein, carbs):
  nutrients = np.array([float(fat)*9, float(protein)*4, float(carbs)*4])  
  nutrients /= np.linalg.norm(nutrients)
  return nutrients

In [0]:
import csv

def get_food_nutrients(nutrient_file):
  foods = {}
  with open(foods_txt) as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    line_count = 0
    for row in csv_reader:
      if line_count == 0:
        print(f'Column names are {", ".join(row)}')
        line_count += 1
      else:        
        nutrients = get_nutrient_array(row[1],row[2],row[3])
        foods[row[0]] = nutrients
        line_count += 1
    print(f'Processed {line_count} lines.')
  return foods

In [80]:
food_nutrients = get_food_nutrients(foods_txt)
print(food_nutrients)

Column names are name, fat, protein, carbs
Processed 102 lines.
{'apple_pie': array([0.58792538, 0.04513367, 0.80765506]), 'baby_back_ribs': array([0.88610838, 0.45289984, 0.09845649]), 'baklava': array([0.63449807, 0.07690886, 0.76908857]), 'beef_carpaccio': array([0.58430473, 0.81153434, 0.        ]), 'beef_tacos': array([0.65023083, 0.26676137, 0.71136365]), 'beef_tartare': array([0.94727127, 0.31575709, 0.05453986]), 'beet_salad': array([0.97348113, 0.09454385, 0.20831695]), 'beignets': array([0.42745936, 0.1341049 , 0.89403265]), 'bibimbap': array([0.2970349 , 0.21602538, 0.93010929]), 'bread_pudding': array([0.22121196, 0.10370363, 0.96969626]), 'breakfast_burrito': array([0.63018909, 0.29008704, 0.7202161 ]), 'bruschetta': array([0.86706091, 0.10223847, 0.48759888]), 'caesar_salad': array([0.96329712, 0.10703301, 0.24617593]), 'cannoli': array([0.8716937 , 0.15496777, 0.46490331]), 'caprese_salad': array([0.94681046, 0.12376607, 0.29703858]), 'carrot_cake': array([0.67741557, 0.

In [0]:
def load_images(food_w_nutrients, directory):
  X = []
  Y = []
  mb = master_bar(listdir(directory))
  for food_group in mb: 
    try:
      for pic in progress_bar(listdir(directory + food_group),
                              parent=mb, comment='food = ' + food_group):
        filename = directory + food_group + '/' + pic
        image, img_path = load_image(filename)
        Y.append(food_w_nutrients[food_group])
        X.append(image)
    except:
      continue
  return X,Y


In [78]:
X, Y = load_images(food_nutrients, images_folder)
print(len(X), len(Y))

4776 4776
